In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib
from numpy.linalg import inv
import iminuit

import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [119]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [111]:
# -------------------------------------
# LS fit with multiple scattering
# ------------------------------------

import numpy as np
class chi2_track_scattering:
    def __init__(self, hits):
        self.hits=hits
        self.func_code = iminuit.util.make_func_code(['x0', 'z0', 't0', 'Ax', 'Az', 'At'])
    def __call__(self, x0, z0, t0, Ax, Az, At):
        chi2_distance = 0

        cov_x, cov_z = self.get_cov(Ax, Az)
        residuals = []

        for hit in self.hits:
            dy = (hit.y - self.hits[0].y)
            model_x = x0 + Ax*dy
            model_z = z0 + Az*dy
            model_t = t0 + At*dy
            residuals.append([model_x-hit.x, model_z-hit.z, model_t-hit.t])
        t_err = [hit.t_err for hit in hits]

        residuals=np.array(residuals)

        chi2_distance = residuals[:,0].T @ np.linalg.inv(cov_x) @residuals[:,0] +\
                        residuals[:,1].T @ np.linalg.inv(cov_z) @residuals[:,1] +\
                        np.sum([(residuals[:,2]/t_err)**2])
        return chi2_distance        

    def get_theta0(self, sin_theta, p=500):
        L_Al =  0.4
        L_Sc = 1.0 # [cm] Scintillator
        L_r_Al = 24.0111/2.7; # [cm] Radiation length Aluminum/ density of Aluminum
        L_r_Sc = 43; # [cm] Radiation length Scintillator (Saint-Gobain paper)

        L_rad = L_Al / L_r_Al + L_Sc / L_r_Sc; # [rad lengths] orthogonal to Layer
        L_rad /= sin_theta; # [rad lengths] in direction of track

        sigma_ms = 13.6 * np.sqrt(L_rad) * (1 + 0.038 * np.log(L_rad)); #
        sigma_ms /= p # [MeV] Divided by 1000 MeV

        return sigma_ms

    def get_cov(self, Ax, Az):

        sin_theta = np.power(Ax**2+Az**2+1, -1/2)
        theta_0 = self.get_theta0(sin_theta)  
        hits = self.hits

        cov_xz = np.zeros((len(self.hits), len(self.hits)))
        for i in range(1,len(self.hits)):
            for j in range(1,len(self.hits)):
                cov_xz[i,j] = np.sum([(hits[i].y-hits[k].y)*(hits[j].y-hits[k].y)*theta_0**2 for k in range(0, min(i,j))])   

        cov_x = cov_xz* Ax**2
        cov_z = cov_xz* Az**2

        cov_x+=np.diag([hit.x_err**2 for hit in hits])
        cov_z+=np.diag([hit.z_err**2 for hit in hits])

        return np.array(cov_x), np.array(cov_z)


def fit_track_scattering(hits, guess):
    x0_init, z0_init,t0_init,Ax_init,Az_init,At_init = guess

    m = iminuit.Minuit(chi2_track_scattering(hits),x0=x0_init, z0=z0_init, t0=t0_init, Ax=Ax_init,Az=Az_init, At=At_init)
    m.limits["x0"]=(-100000,100000)
    m.limits["z0"]=(-100000,100000)
    m.limits["t0"]=(-100,1e5)
    m.limits["Ax"]=(-10,10) # Other
    m.limits["Az"]=(-10,10)
    m.limits["At"]=(0.001,0.2) # Beam direction; From MKS unit to cm/ns = 1e2/1e9=1e-7
    m.errors["x0"]=0.1
    m.errors["z0"]=0.1
    m.errors["t0"]=0.1
    m.errors["Ax"] = 0.0001
    m.errors["At"] = 0.0001
    m.errors["Az"] = 0.0001

    m.migrad()  # run optimiser
    m.hesse()   # run covariance estimator
    
    return m          



def fit_track_ana(hits, scattering = False, iters = 2):
    y0 = hits[0].y

    X = np.array([hit.x for hit in hits])
    Z = np.array([hit.z for hit in hits])
    T = np.array([hit.t for hit in hits])
    H = np.array([[1,hit.y-y0] for hit in hits])

    if scattering is False:
        Vx_inv = np.diag([1/hit.x_err for hit in hits ])
        Vz_inv = np.diag([1/hit.z_err for hit in hits ])
        Vt_inv = np.diag([1/hit.t_err for hit in hits ])
    elif scattering is True:
        if iters==0:
            Param_all, Error_all = fit_track_ana(hits, scattering = False)
        else:
            iters-=1
            Param_all, Error_all = fit_track_ana(hits, scattering = True, iters = iters)

        # chi2_object = chi2_track_scattering(hits)
        Vx, Vz = chi2_track_scattering(hits).get_cov(Param_all[3], Param_all[4])
        Vx_inv = np.linalg.inv(Vx)
        Vz_inv = np.linalg.inv(Vz)
        Vt_inv = np.diag([1/hit.t_err for hit in hits ])



    Error_x = np.linalg.inv(H.T @ Vx_inv @ H)
    Param_x = Error_x @ (H.T @ Vx_inv @ X)

    Error_z = np.linalg.inv(H.T @ Vz_inv @ H)
    Param_z = Error_z @ (H.T @ Vz_inv @ Z)

    Error_t = np.linalg.inv(H.T @ Vt_inv @ H)
    Param_t = Error_t @ (H.T @ Vt_inv @ T)  

    Error_all = np.array([[Error_x[0,0],           0,           0,Error_x[0,1],           0,             0],
                            [           0,Error_z[0,0],           0,           0,Error_z[0,1],             0],
                            [           0,           0,Error_t[0,0],           0,           0,  Error_t[0,1]],
                            [Error_x[1,0],           0,           0,Error_x[0,1],           0,             0],
                            [           0,Error_z[1,0],           0,           0,Error_z[0,1],             0],
                            [           0,           0,Error_t[1,0],           0,           0,  Error_t[0,1]]])
    Param_all = [Param_x[0],Param_z[0],Param_t[0], Param_x[1],Param_z[1],Param_t[1]]

    return  Param_all, Error_all

In [112]:
x = [241.62934581783023, 259.25, 221.49664808213277, 227.75]
y = [9894.0, 9975.599999999999, 10057.2, 10138.8]
z = [12456.75, 12450.5, 12461.25, 12450.5]
t = [40.91365706617539, 45.79668867914941, 48.189471653643565, 50.295569678622876]
layers = [0,1,2,3,]

hits = Util.hit.make_hits(x,y,z,t,layers)


# # --=-------------------------
events=joblib.load("events_example.joblib")
events
hits_ind = [18, 19, 20, 21]
hits = [events[21][i] for i in hits_ind]
hits

# hits

[Hit(x=1410.0451781469023, y=9893.0, z=10110.25, t=39.05387203461752, x_err=13.416789873441976, y_err=0.2886751345948129, z_err=1.299038105676658, t_err=0.7071067811865475, layer=2, ind=18),
 Hit(x=1421.25, y=9974.2, z=10106.435975778155, t=41.1256624916363, x_err=1.299038105676658, y_err=0.2886751345948129, z_err=13.416789873441976, t_err=0.7071067811865475, layer=3, ind=19),
 Hit(x=1394.9714154790188, y=10055.400000000001, z=10114.75, t=45.28472145915637, x_err=13.416789873441976, y_err=0.2886751345948129, z_err=1.299038105676658, t_err=0.7071067811865475, layer=4, ind=20),
 Hit(x=1398.75, y=10136.599999999999, z=10114.668345176091, t=47.621266524957505, x_err=1.299038105676658, y_err=0.2886751345948129, z_err=13.416789873441976, t_err=0.7071067811865475, layer=5, ind=21)]

In [118]:
# %timeit -n 1 fit_ls = fit_track_scattering(hits,guess)
# %timeit -n 1 Util.track.run_kf(hits, multiple_scattering=True)

In [114]:
guess = Util.track.guess_track(hits)
fit_ls = fit_track_scattering(hits,guess)
fit_ls.values, fit_ls.errors, fit_ls.fval

(<ValueView x0=1431.917738097749 z0=10110.23233353367 t0=38.79219263164063 Ax=-0.13601406046215025 Az=0.027450162553371626 At=0.036774917451006736>,
 <ErrorView x0=2.0307719975171494 z0=1.2960167608980555 t0=0.5916167617324533 Ax=0.011282232668802017 Az=0.011209905463264391 At=0.003892803656654807>)

In [129]:

values, cov, chi2 = Util.track.fit_track_ana(hits, scattering = True, iters = 2)
print(values)
print(np.sqrt(np.diag(cov)))
print(chi2)

[1431.917111507736, 10110.232307722974, 38.79219426181089, -0.13601359402468205, 0.027449509218882895, 0.03677492911150271]
[2.03085772 1.29601817 0.70354442 0.01128354 0.01120993 0.00463128]


In [73]:
kalman_result = Util.track.run_kf(hits, multiple_scattering=True)
kalman_result.Xsm[0], np.sqrt(np.diag(kalman_result.Csm[0]))

# propagate the KF result from the second hit to the first hit
Ax, Az, At = kalman_result.Xsm[0][3:]
velocity = [Ax, Az, At]

mi, Vi, Hi, Fi, Qi = Util.track.add_measurement(hits[0], hits[0].y - hits[1].y, velocity=velocity)
Xp_i = Fi@kalman_result.Xsm[0]
Cp_i = Fi@kalman_result.Csm[0]@Fi.T + Qi 

rp_i = mi - Hi@Xp_i
Rp_i = Vi + Hi@Cp_i@Hi.T
# Kalman Gain K
K = Cp_i.dot(Hi.T).dot(inv(Rp_i))
# Filtered State
Xf = Xp_i + K@rp_i# Combination of the predicted state, measured values, covariance matrix and Kalman Gain
Cf = (np.identity(len(Xf)) - K@Hi).dot(Cp_i)
state_predicted_step_0 = Xf
statecov_predicted_step_0 = Cf 


# Add the covariance of one additional layer:
mi, Vi, Hi, Fi, Qi = Util.track.add_measurement(hits[0], -1.5, velocity=velocity)
cov = statecov_predicted_step_0 + Qi
chi2 = kalman_result.chift_total



x0 = state_predicted_step_0[0]
z0 = state_predicted_step_0[1]
t0 = state_predicted_step_0[2]
Ax = state_predicted_step_0[3]
Az = state_predicted_step_0[4]
At = state_predicted_step_0[5]

print(x0,z0,t0,Ax,Az,At)
print(np.sqrt(np.diag(cov)))
print(chi2)

1431.316053291954 10110.240427295634 38.90108926607831 -0.1321921675787688 0.02741550337342524 0.03617869007763045
[2.14791128 0.95190763 0.45375471 0.01609054 0.01298649 0.00333938]


In [74]:
kf = Util.track.run_kf(hits[::-1], multiple_scattering=True)
print(kf.Xf[-1])
print(np.sqrt(np.diag(kf.Cf[-1])))
print(kf.chift_total)


[ 1.43187611e+03  1.01102323e+04  3.87926623e+01 -1.35501699e-01
  2.74642672e-02  3.67618891e-02]
[2.11014624 1.29602267 0.59161323 0.01332928 0.01167729 0.00389475]
